# Example of transform to tidy format
This notebook gives a short example of how one data file can be read from file with Pandas and transformed to a tidy format. I haven't done anything about formatting emission names. Operations between data sets might be easier with a multiindex to allow for easy matching of emissions/countries/years.

In [1]:
import pandas as pd
from os.path import join

## Import one data file

In [2]:
fn = join('inputs', 'emission_data_EDGAR_FAO.csv')
df = pd.read_csv(fn, sep=';', header=[0, 1], index_col=0)
df = df.iloc[:, :-1] # drop last column

In [3]:
df.head()

Countries,CO2 total excl. Land Use excl. BB (ktCO2),Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,CH4 ( ktCH4),Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,Unnamed: 10_level_0,...,SF6 tot (ktSF6),Unnamed: 27_level_0,Unnamed: 28_level_0,Unnamed: 29_level_0,Unnamed: 30_level_0,CO2 Land Use (ktCO2),Unnamed: 32_level_0,Unnamed: 33_level_0,Unnamed: 34_level_0,Unnamed: 35_level_0
,1990,1995,2000,2005,2010,1990,1995,2000,2005,2010,...,1990,1995,2000,2005,2010,1990,1995,2000,2005,2010
Afghanistan,2013.73,2305.02,2456.66,3074.88,6357.08,363.22,375.52,446.87,509.46,642.54,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Albania,6486.03,2065.37,3137.54,4161.30,4092.34,121.28,131.84,124.23,117.96,123.45,...,0.00,0.0,0.00,0.00,0.00,121.15,121.15,121.34,889.36,-211.04
Algeria,67632.52,80029.04,84056.33,97923.23,116212.87,1538.07,1885.86,2145.37,2225.59,2267.88,...,0.01,0.0,0.01,0.01,0.01,880.00,880.00,880.00,513.33,-6746.67
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-19.46,-19.46,-19.46,-28.92,-38.94
Angola,11285.76,13025.84,16601.29,16199.45,24409.17,1050.30,1020.02,750.40,778.98,885.83,...,0.00,0.0,0.00,0.00,0.00,83384.43,83384.43,82397.21,94767.62,81040.65


## Create a new multiindex for columns

In [7]:
iterables = [['CO2 total excl. Land Use excl. BB (ktCO2)',
              'CH4 ( ktCH4)',
              'N2O (ktN2O)',
              'HFC tot (GWP100, ktCO2e)',
              'PFC tot (GWP100, ktCO2e)',
              'SF6 tot (ktSF6)',
              'CO2 Land Use (ktCO2)'], 
             [1990,1995,2000,2005,2010]]

idx = pd.MultiIndex.from_product(iterables, names=['emission', 'year'])

df.columns = idx

In [8]:
df.head()

emission    CO2 total excl. Land Use excl. BB (ktCO2)                      \
year                                             1990      1995      2000   
Afghanistan                                   2013.73   2305.02   2456.66   
Albania                                       6486.03   2065.37   3137.54   
Algeria                                      67632.52  80029.04  84056.33   
Andorra                                           NaN       NaN       NaN   
Angola                                       11285.76  13025.84  16601.29   

emission                         CH4 ( ktCH4)                             \
year             2005       2010         1990     1995     2000     2005   
Afghanistan   3074.88    6357.08       363.22   375.52   446.87   509.46   
Albania       4161.30    4092.34       121.28   131.84   124.23   117.96   
Algeria      97923.23  116212.87      1538.07  1885.86  2145.37  2225.59   
Andorra           NaN        NaN          NaN      NaN      NaN      NaN   
Angola       16199.45   24409.17      1050.30  1020.02   750.40   778.98   

emission                ...    SF6 tot (ktSF6)                         \
year            2010    ...               1990 1995  2000  2005  2010   
Afghanistan   642.54    ...               0.00  0.0  0.00  0.00  0.00   
Albania       123.45    ...               0.00  0.0  0.00  0.00  0.00   
Algeria      2267.88    ...               0.01  0.0  0.01  0.01  0.01   
Andorra          NaN    ...                NaN  NaN   NaN   NaN   NaN   
Angola        885.83    ...               0.00  0.0  0.00  0.00  0.00   

emission    CO2 Land Use (ktCO2)                                          
year                        1990      1995      2000      2005      2010  
Afghanistan                 0.00      0.00      0.00      0.00      0.00  
Albania                   121.15    121.15    121.34    889.36   -211.04  
Algeria                   880.00    880.00    880.00    513.33  -6746.67  
Andorra                   -19.46    -19.46    -19.46    -28.92    -38.94  
Angola                  83384.43  83384.43  82397.21  94767.62  81040.65  

[5 rows x 35 columns]

## Melt the df to tidy, re-add country names for the index

In [9]:
df2 = df.melt()
index = list(df.index) * 5 * 7 # 5 years and 7 emission types

df2.index = index
df2.head()

,emission,year,value
Afghanistan,CO2 total excl. Land Use excl. BB (ktCO2),1990,2013.73
Albania,CO2 total excl. Land Use excl. BB (ktCO2),1990,6486.03
Algeria,CO2 total excl. Land Use excl. BB (ktCO2),1990,67632.52
Andorra,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Angola,CO2 total excl. Land Use excl. BB (ktCO2),1990,11285.76


## Easily slice null values

In [10]:
df2.loc[df2['value'].isnull()]

,emission,year,value
Andorra,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
European Union,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Liechtenstein,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Marshall Islands,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
"Micronesia, Federated States of",CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Monaco,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Montenegro,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Niue,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
San Marino,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
Serbia,CO2 total excl. Land Use excl. BB (ktCO2),1990,NaN
